# Train a Model with SageMaker Autopilot

Customer review를 예측하기 위해 SageMaker Autopilot을 사용합니다.  
Autopilot는 AutoML에 대해 white-box 접근 방식으로 구현합니다.

<img src="./img/autopilot.png" width="80%" align="left">

# Introduction

Amazon SageMaker Autopilot은 데이터 세트에서 자동 기계 학습 (AutoML)을 수행하는 서비스입니다. Autopilot은 UI 또는 AWS SDK를 통해 사용할 수 있습니다. 이 노트북에서는 AWS SDK를 사용하여 텍스트 처리 및 sentiment classification 기계 학습 파이프 라인을 생성과 배포를 합니다.

# Setup

* 모델 학습에 사용되는 S3 bucket과 prefix 가 필요합니다. 참고 : 필요한 dataset은 동일 region에 있어야 합니다.
* 이 노트북의 IAM role은 dataset에 액세스가 가능해야 합니다.

In [ ]:
%store -r

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
role = sagemaker.get_execution_role()
sm = boto3.Session().client(service_name='sagemaker', region_name=region_name)

# Dataset

In [ ]:
print(header_train_s3_uri)

In [ ]:
!aws s3 ls $header_train_s3_uri

# Setup the S3 Location for the Autopilot-Generated Assets 

* Jupyter Notebooks (Analysis)
* Python Scripts (Feature Engineering)
* Trained Models.

In [ ]:
prefix_model_output = 'models/autopilot'

model_output_s3_uri = 's3://{}/{}'.format(job_bucket, prefix_model_output)

print(model_output_s3_uri)


In [ ]:
max_candidates = 3

job_config = {
    'CompletionCriteria': {
      'MaxRuntimePerTrainingJobInSeconds': 600,
      'MaxCandidates': max_candidates,
      'MaxAutoMLJobRuntimeInSeconds': 3600
    },
}

input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': '{}'.format(header_train_s3_uri)
        }
      },
      'TargetAttributeName': 'star_rating'
    }
]

output_data_config = {
    'S3OutputPath': '{}'.format(model_output_s3_uri)
}

# Launch the SageMaker Autopilot job

`create_auto_ml_job` API를 이용하여 Autopilot job을 실행합니다.

In [ ]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-dm-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

`ProblemType`를 특정할 수 없는 경우에는 Autopilot이 자동으로 regression 또는 Classification (binary 또는 multi-class) 를 탐지합니다.

In [ ]:
sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig=job_config,
#                      ProblemType="Classification",
                      RoleArn=role)

# Tracking the progress of the Autopilot job

SageMaker Autopilot은 high-level 단계로 아래와 같이 구성됩니다.

* _Data Analysis_ 데이터를 요약하고 분석하여 탐색할 feature engineering 기법, 하이퍼파라미터와 탐색 모델을 결정합니다.
* _Feature Engineering_ 데이터가 클린징, 밸런싱, 결합 과 훈련/검증데이터셋으로 분리합니다.
* _Model Training and Tuning_ 가장 높은 성능의 features, 하이퍼라미터와 모델을 선택하고 학습힙니다.

# Analyzing Data

In [ ]:
# Sleep for a bit to ensure the AutoML job above has time to start
import time
time.sleep(3)

job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']

if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('AnalyzingData'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Data analysis complete")
    
print(job)

In [ ]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print(job)

# Feature Engineering

In [ ]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('FeatureEngineering'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Feature engineering complete")
    
print(job)

# Model Training and Tuning

In [ ]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('ModelTuning'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Model tuning complete")
    
print(job)

<h2><span style="color:red">위 Autopilot이 완료되기 전까지 기다려 주시기 바랍니다.</span></h2>

# View Generated Notebooks

데이터 분석이 완료되며, SageMaker AutoPilot 2가지 노트북을 생성합니다.
* Data exploration,
* Candidate definition.

## Copy the Generated Notebooks Locally

In [ ]:
generated_resources = job['AutoMLJobArtifacts']['DataExplorationNotebookLocation'].rstrip('notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb')
generated_resources

In [ ]:
!aws s3 cp --recursive $generated_resources .

## In the file view, open the following folders:
```
notebooks/
generated_module/
```
이 폴더에서 많은 정보를 보실 수 있습니다.

# Viewing All Candidates

model tuning이 완료되면 AutoML에서 탐색 한 모든 후보(서로 다른 하이퍼 파라미터 조합을 가진 파이프 라인 평가)를 보고, 최종 성능 메트릭별로 정렬할 수 있습니다.
Once model tuning is complete, you can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by AutoML and sort them by their final performance metric.

In [ ]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, 
                                                SortBy='FinalObjectiveMetricValue')['Candidates']
for index, candidate in enumerate(candidates):
    print(str(index) + "  " 
        + candidate['CandidateName'] + "  " 
        + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))

# Inspect Trials using Experiments API

SageMaker Autopilot은 자동으로 새로운 experiment를 생성하고, 각 trial에 대한 정보를 experiment에 추가합니다.

In [ ]:
from sagemaker.analytics import ExperimentAnalytics, TrainingJobAnalytics

exp = ExperimentAnalytics(
        sagemaker_session=sess, 
        experiment_name=auto_ml_job_name + '-aws-auto-ml-job',
)

df = exp.dataframe()
print(df)

# Explore the Best Candidate

Dataset에서 AutoML job을 완료하고 trials를 시각화하면, 단일 API call로 어떤 trials에 대해 모델을 생성할 수 있습니다. [Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html) 사용하여 온라인으로 또는 batch prediction으로 모델을 배포합니다. 이 노트북에서는 가장 성능이 좋은 trial을 선택하여 inference로 배포합니다.


In [ ]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_identifier = best_candidate['CandidateName']

print("Candidate name: " + best_candidate_identifier)
print("Metric name: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("Metric value: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

In [ ]:
best_candidate

inference 파이프라인은 model과 containers로 구성되어 있습니다.

In [ ]:
for container in best_candidate['InferenceContainers']:
    print(container['Image'])
    print(container['ModelDataUrl'])
    print('======================')

# Autopilot Chooses XGBoost as Best Candidate!


Autopilot은 XGBoost 모델의 기본적인 값과 다른 하이퍼파라미터와 feature transformations을 선택합니다.

# Deploy the Model as a REST Endpoint

Batch transformations 또한 지원하지만, 여기서는 REST Endpoint를 생성합니다.

In [ ]:
model_name = 'automl-dm-model-' + timestamp_suffix

model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Best candidate model ARN: ', model_arn['ModelArn'])

In [ ]:
# EndpointConfig name
timestamp_sufafix = strftime('%d-%H-%M-%S', gmtime())
epc_name = 'automl-dm-epc-' + timestamp_suffix

# Endpoint name
autopilot_endpoint_name = 'automl-dm-ep-' + timestamp_suffix
variant_name = 'automl-dm-variant-' + timestamp_suffix

print(autopilot_endpoint_name)
print(variant_name)

In [ ]:
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType':'ml.m5.large',
                                                           'InitialInstanceCount': 1,
                                                           'ModelName': model_name,
                                                           'VariantName': variant_name}])


In [ ]:
create_endpoint_response = sm.create_endpoint(EndpointName=autopilot_endpoint_name,
                                              EndpointConfigName=epc_name)
print(create_endpoint_response['EndpointArn'])

# Wait for the Model to Deploy


모델을 deploy하는데에는 5~10분 정도 소요됩니다.

In [ ]:
sm.get_waiter('endpoint_in_service').wait(EndpointName=autopilot_endpoint_name)


In [ ]:
resp = sm.describe_endpoint(EndpointName=autopilot_endpoint_name)
status = resp['EndpointStatus']

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

# Test Our Model with Some Example Reviews
Let's do some ad-hoc predictions on our model.

In [ ]:
sm_runtime = boto3.client('sagemaker-runtime')

In [ ]:
csv_line_predict_positive = """I loved it!"""

response = sm_runtime.invoke_endpoint(EndpointName=autopilot_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_positive)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

In [ ]:
csv_line_predict_meh = """It's OK."""

response = sm_runtime.invoke_endpoint(EndpointName=autopilot_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_meh)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

In [ ]:
csv_line_predict_negative = """The worst product ever."""

response = sm_runtime.invoke_endpoint(EndpointName=autopilot_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_negative)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

# Create an Athena Table with Sample Reviews

In [ ]:
table_name = 'product_reviews'

In [ ]:
# Create Table SQL Statement
statement = """
CREATE TABLE IF NOT EXISTS {}.{} AS 
SELECT review_id, review_body 
FROM {}.{}
""".format(database_name, table_name, database_name, table_name_tsv)

print(statement)

In [ ]:
# Execute statement using connection cursor
from pyathena import connect
from pyathena.util import as_pandas

cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

In [ ]:
statement = 'SELECT * FROM {}.{} LIMIT 10'.format(database_name, table_name)
cursor.execute(statement)

In [ ]:
df_show = as_pandas(cursor)
df_show

### Preview Feature를 사용하기 위해  `AmazonAthenaPreviewFunctionality` Work Group에 추가합니다. 

In [ ]:
import boto3
from botocore.exceptions import ClientError

client = boto3.client('athena')

try:
    response = client.create_work_group(Name='AmazonAthenaPreviewFunctionality') 
    print(response)
except ClientError as e:
    if e.response['Error']['Code'] == 'InvalidRequestException':
        print("Workgroup already exists.")
    else:
        print("Unexpected error: %s" % e)
    


# SQL Query 생성하기

`USING FUNCTION`절은 Athena 함수(preview) 또는 쿼리 내 후속 `SELECT` 문에서 참조할 수 있는 여러 함수에서 ML을 지정하여 사용합니다. 변수와 return 값에 대한 데이터 타입과 변수명, 함수명을 정의합니다.

In [ ]:
statement = """
USING FUNCTION predict_star_rating(review_body VARCHAR) 
    RETURNS VARCHAR TYPE
    SAGEMAKER_INVOKE_ENDPOINT WITH (sagemaker_endpoint = '{}'
)
SELECT review_id, review_body, predict_star_rating(REPLACE(review_body, ',', ' ')) AS predicted_star_rating 
    FROM {}.{} LIMIT 10
    """.format(autopilot_endpoint_name, database_name, table_name)

print(statement)

In [ ]:
# Execute statement using connection cursor
cursor = connect(region_name=region_name, 
                 s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement, 
               work_group='AmazonAthenaPreviewFunctionality')

In [ ]:
df = as_pandas(cursor)
df

In [ ]:
sm.delete_endpoint(
    EndpointName=autopilot_endpoint_name
)

In [ ]:
%store